# Generating Simulated Nanoparticle Images

In this notebook, simulated nanoparticles will be generated. These will then be rotated using rotation matrices and projected onto a 2D plane. This projection will be blurred and Poisson noise will be added. The projections will be saved as images directly into Google Drive, separated into training, validation, and testing datasets.

All of my work can be found at https://github.com/javidahmed64592/Y4-Nanoparticles-Project.

# Importing the relevant libraries and configuring the figure.

In [ ]:
import warnings
warnings.simplefilter("ignore", DeprecationWarning)

import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import datetime

img_width, img_height = 128, 128
plt.style.use("dark_background")
fig = plt.figure(figsize=(3, 3))

<Figure size 216x216 with 0 Axes>

The images are saved directly in Google Drive.

In [ ]:
from google.colab import drive
drive_path = "/content/drive"
drive.mount(drive_path, force_remount=True)

cwd = os.path.join(drive_path, "MyDrive", "Nanoparticles")
file_type = "png"

Mounted at /content/drive


The following functions are used to augment the data by randomly adjusting the brightness and channels.

In [ ]:
def brightness(img, low, high):
  """
  Adjusts image brightness randomly by a factor between low and high.

  Inputs:
    img: NumPy array, image to use to adjust brightness
    low: Float, lower limit for random factor to multiply image's brightness
    high: Float, upper limit for random factor to multiply image's brightness
  
  Outputs:
    img: NumPy array, image after brightness has been adjusted
  """
  value = np.random.uniform(low, high)
  hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
  hsv = np.array(hsv, dtype = np.float64)
  hsv[:,:,1] = hsv[:,:,1] * value
  hsv[:,:,1][hsv[:,:,1] > 255]  = 255
  hsv[:,:,2] = hsv[:,:,2] * value 
  hsv[:,:,2][hsv[:,:,2] > 255]  = 255
  hsv = np.array(hsv, dtype = np.uint8)
  img = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)
  return img

In [ ]:
def channel_shift(img, value):
  """
  Adjusts image channels randomly by a specified value.

  Inputs:
    img: NumPy array, image to use to adjust channels
    value: Integer, adjust channels by some random integer in the range [-value, value)
  
  Outputs:
    img: NumPy array, image after channels have been adjusted
  """
  value = int(np.random.uniform(-value, value))
  img = img + value
  img[:,:,:][img[:,:,:] > 255]  = 255
  img[:,:,:][img[:,:,:] < 0]  = 0
  img = img.astype(np.uint8)
  return img

The following function is used to map a random Poisson array to be in the limits of the plot to simulate Poisson background noise.

In [ ]:
def map_params(array_to_map, map_range):
  """
  Maps an array from its range to a new specified range.

  Inputs:
    array_to_map: NumPy array, array to be mapped to a new range
    map_range: NumPy array, range to use for new mapped array
  
  Outputs:
    mapped_array: NumPy array, array_to_map mapped to new map_range
  """
  y_max, y_min = np.max(array_to_map), np.min(array_to_map)
  map_max, map_min = np.max(map_range), np.min(map_range)

  mapped_array = map_min + ((array_to_map - y_min) * (map_max - map_min) / (y_max - y_min))

  return mapped_array

# Creating the class.

The shape3D class generates a lattice to simulate different nanoparticle shapes. These bodies can be rotated in 3 dimensions and then projected onto a 2D plane. This plane can then have noise added to simulate Poisson noise from measuring instruments. This is then saved as an image. These functions will be shared for all lattice shapes.

In [ ]:
class shape3D:
  """
  This class creates a 3D shape from a specified width and 3 angles of rotation, rx, ry, rz. The
  shape can then be projected onto 2 axes after being rotated. These projections are saved as
  images, and they can also have a Gaussian blur applied to them and those are saved in a
  subfolder.
  """
  def __init__(self, width=10, rx=0, ry=0, rz=0, a=1, defect=0, pos_error=0):
    """
    Initialises the class.

    Inputs:
      width: Integer, width of the shape being created
      rx, ry, rz: Float, rotations in degrees about the x, y, z axes respectively
      a: Float, lattice spacing
      defect: Float between 0 and 1, percentage of defects in the shape, 0 is no defects
      pos_error: Float, error in the position of each point
    """
    self.width = width
    self.rx = rx
    self.ry = ry
    self.rz = rz
    self.defect = defect
    self.pos_error = pos_error
    self.a = a
    self.alpha = 0.25

    data = "W%s RX%s RY%s RZ%s D%s" % (self.width, self.rx, self.ry, self.rz, int(100*self.defect))
    self.name = "%s %s" % (self.name, data)

    self.generate() # Generating the shape
    self.rotate() # Rotating the shape
    self.projection2D() # Projecting the shape onto a 2D surface

  def generate(self):
    """
    Subclasses override this function to generate self.coords.
    """
    pass

  def rotate(self):
    """
    Calculates the rotation matrix to rotate the shape.
    """
    # Converting degrees to radians
    rx = np.deg2rad(self.rx)
    ry = np.deg2rad(self.ry)
    rz = np.deg2rad(self.rz)

    # Rotation matrices
    Rx = np.array([[           1,           0,           0],
                    [           0,  np.cos(rx), -np.sin(rx)],
                    [           0,  np.sin(rx),  np.cos(rx)]])

    Ry = np.array([[  np.cos(ry),           0,  np.sin(ry)],
                    [           0,           1,           0],
                    [ -np.sin(ry),           0,  np.cos(ry)]])

    Rz = np.array([[  np.cos(rz), -np.sin(rz),           0],
                    [  np.sin(rz),  np.cos(rz),           0],
                    [           0,           0,           1]])

    # Applying the rotation
    self.R = np.dot(Rz, np.dot(Ry, Rx))
    self.coords = np.matmul(self.R, self.coords)

    self.rotation = self.R[:,0:2].T.reshape((1, 6))

  def projection2D(self):
    """
    Projects 3D shape onto 2D plane.
    """
    # Removing z axis
    self.coords2D = self.coords[0:, :]
    coords2D_shape = np.shape(self.coords2D)

    # Adding defects to the shape
    if self.defect != 0:
        iters = int(self.defect * coords2D_shape[0])

        for _ in range(iters):
            self.coords2D = np.delete(self.coords2D, np.random.randint(coords2D_shape[0]-1), 0)

    # Translating the shape around randomly
    self.coords2D[0, :] += np.random.uniform(-self.a, self.a) * 2
    self.coords2D[1, :] += np.random.uniform(-self.a, self.a) * 2

    # Creating the plot
    self.ax = fig.add_subplot(1, 1, 1)
    self.ax.set_axis_off()

    lim = 10
    self.lims = [-lim, lim]
    self.ax.set_xlim(self.lims)
    self.ax.set_ylim(self.lims)
    self.ax.set_aspect('equal', adjustable='box')
    self.ax.scatter(self.coords2D[0, :], self.coords2D[1, :], s=2, c="white", alpha=self.alpha)

  def save_projection(self, save_path=os.path.join(os.getcwd(), "Simulated Data"), file_name = "default", file_type=".png", blur=4, iter=0, aug=False):
    """
    Saves the 2D projection.
    
    Inputs:
      save_path: String, folder in which to save the projections
      file_name: String, desired name for the projection's file name
      file_type: String, what file type to save the file as
      blur: Integer, strength of Gaussian blur
      iter: Integer, number of cubes with the same properties + 1
    """
    if not os.path.exists(save_path):
      os.makedirs(save_path)

    if file_name == "default":
      file_name = self.name + (" %s" % iter)

    # Creating the image from the canvas
    fig.canvas.draw()
    img = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep='')
    img = img.reshape(fig.canvas.get_width_height()[::-1] + (3, ))
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    # Cropping the outer borders
    factor = 0.15
    low_y = int(img.shape[0] * factor)
    high_y = int(img.shape[0] - low_y)
    low_x = int(img.shape[1] * factor)
    high_x = int(img.shape[1] - low_x)

    cropped = img[low_x:high_x, low_y:high_y]

    # Blurring the projection
    kernel = np.ones((blur,blur),np.float32)/blur**2
    dst = cv2.filter2D(cropped,-1,kernel)

    dst = cv2.resize(dst, (img_width, img_height))

    # Where to save
    blur_name = file_name + (" B%s" % (blur))
    rotation6d = self.rotation.tolist()[0]
    blur_path = os.path.join(save_path, "%s_%s" % (self.name.split()[0], rotation6d))
    blur_file_name = os.path.join(blur_path, blur_name)

    if not os.path.exists(blur_path):
      os.makedirs(blur_path)

    # Generating noise over the blurred image
    plt.cla()
    self.ax.imshow(dst, extent=[0, img_width, 0, img_height])
    noise_param = 8000
    noise = map_params(np.random.poisson(10000, (noise_param, 2)), 3 * np.array(self.lims))
    self.ax.scatter(noise[:,0], noise[:,1], s=1, c="white", alpha=0.035)

    # Converting the canvas to image then saving
    fig.canvas.draw()
    dst = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep='')
    dst = dst.reshape(fig.canvas.get_width_height()[::-1] + (3, ))
    dst = cv2.resize(dst[low_x:high_x, low_y:high_y], (img_width, img_height))

    # Random augmentations
    if aug:
      dst_1 = brightness(dst, 1, 2.2)
      dst_2 = channel_shift(dst, 20)
      dst_1 = cv2.cvtColor(dst_1, cv2.COLOR_RGB2GRAY)
      dst_2 = cv2.cvtColor(dst_2, cv2.COLOR_RGB2GRAY)
      cv2.imwrite("%s augbright.%s" % (blur_file_name, file_type), dst_1)
      cv2.imwrite("%s augchannel.%s" % (blur_file_name, file_type), dst_2)

    dst = cv2.cvtColor(dst, cv2.COLOR_RGB2GRAY)
    cv2.imwrite("%s.%s" % (blur_file_name, file_type), dst)

    fig.clf()

Creating the subclasses for specific shapes which inherits from the shape3D class.

In [ ]:
class cube3D(shape3D):
  """
  This class inherits from the shape3D class and is specifically for a cube.
  """
  def __init__(self, width=10, rx=0, ry=0, rz=0, a=1, defect=0, pos_error=0):
    """
    Create a cube and include that in the object name.
    """
    self.name = "Cube"
    super().__init__(width, rx, ry, rz, a, defect, pos_error)

  def generate(self):
    """
    Generates all xyz coordinates.
    """
    x0 = self.a * np.arange(-(self.width-1)/2, self.width/2, 1)
    self.coords = np.reshape(np.meshgrid(x0, x0, x0), (3, -1))
    self.coords += np.random.normal(0, self.pos_error, np.shape(self.coords)) * self.a

class tetra3D(shape3D):
  """
  This class inherits from the shape3D class and is specifically for a tetrahedron.
  """
  def __init__(self, width=10, rx=0, ry=0, rz=0, a=1, defect=0, pos_error=0):
    """
    Create a tetrahedron and include that in the object name.
    """
    if width < 2:
      width = 2
      print("Width must be greater than or equal to 2, setting width to 2.")
    if width > 19:
      width = 19
      print("Width must be less than or equal to 19, setting width to 19.")

    self.name = "Tetrahedron"
    super().__init__(width, rx, ry, rz, a, defect, pos_error)

  def generate(self):
    """
    Generates all xyz coordinates.
    """
    shape_folder = os.path.join(cwd, "Vertices", "Tetrahedron")
    shape_txt = "Tetra%s_Verts.txt" % self.width
    self.coords = np.loadtxt(os.path.join(shape_folder, shape_txt))
    self.coords *= self.a
    self.coords += np.random.normal(0, self.pos_error, np.shape(self.coords)) * self.a

class octa3D(shape3D):
  """
  This class inherits from the shape3D class and is specifically for a octahedron.
  """
  def __init__(self, width=10, rx=0, ry=0, rz=0, a=1, defect=0, pos_error=0):
    """
    Create a octahedron and include that in the object name.
    """
    if width < 2:
      width = 2
      print("Width must be greater than or equal to 2, setting width to 2.")
    if width > 19:
      width = 19
      print("Width must be less than or equal to 19, setting width to 19.")

    self.name = "Octahedron"
    super().__init__(width, rx, ry, rz, a, defect, pos_error)

  def generate(self):
    """
    Generates all xyz coordinates.
    """
    shape_folder = os.path.join(cwd, "Vertices", "Octahedron")
    shape_txt = "Octa%s_Verts.txt" % self.width
    self.coords = np.loadtxt(os.path.join(shape_folder, shape_txt))
    self.coords *= self.a
    self.coords += np.random.normal(0, self.pos_error, np.shape(self.coords)) * self.a

class torus3D(shape3D):
  """
  This class inherits from the shape3D class and is specifically for an arbitrarily modified torus (to remove symmetries).
  """
  def __init__(self, width=10, rx=0, ry=0, rz=0, a=1, defect=0, pos_error=0):
    """
    Create a modified torus and include that in the object name.
    """
    self.name = "Torus"
    super().__init__(width, rx, ry, rz, a, defect, pos_error)

  def generate(self):
    """
    Generates all xyz coordinates.
    """
    shape_folder = os.path.join(cwd, "Vertices", "Misc Shapes")
    shape_txt = "Torus_Verts.txt"
    self.coords = np.loadtxt(os.path.join(shape_folder, shape_txt))
    self.coords *= self.a
    self.coords += np.random.normal(0, self.pos_error, np.shape(self.coords)) * self.a

# Generating the images.

The following function is used to generate a list of numbers in a specified range and with a specified step between each number.

In [ ]:
def generate_params(start=0, end=None, step=None, scale=1):
  """
  Adjusts image brightness randomly by a factor between low and high.

  Inputs:
    start: Integer, lower end of desired range
    end: Integer, higher end of desired range
    step: Integer, step between each number
    scale: Float or Integer, scale to multiply every number
  
  Outputs:
    List, list of numbers between start and end spaced evenly by specified step
  """
  if end is None:
    return [start * scale]

  if step is None:
    return [start * scale, end * scale]

  return [scale * (start + i * step) for i in range(1 + int((end-start)/step))]

Configuring the properties of the lattices to be generated.

In [ ]:
# Properties of the shape and its projection
shape_class = tetra3D
width = generate_params(13)
dr_train = 1
rx_train = generate_params(0, 45, dr_train) # Angle in degrees
ry_train = generate_params(0, 45, dr_train)

dr_test = 1
rx_test = generate_params(0, 45, dr_test)
ry_test = generate_params(0, 45, dr_test)

rz = generate_params(0)

a = 1 # Pt nanoparticle lattice spacing: 0.24nm, atom size is ~0.1nm | 0.75 for cube
d = generate_params(5, 15, 10, scale=1/100) # Percentage defect of the shape
blur = 25 # Strength of Gaussian blur

iters = 1
iters_train = 4 * iters
iters_valid = iters
iters_test = iters

In [ ]:
num_rots_train = len(rx_train) * len(ry_train) * len(rz)
num_imgs_train = len(width) * len(d) * num_rots_train * iters_train
num_imgs_valid = len(width) * len(d) * num_rots_train * iters_valid

num_rots_test = len(rx_test) * len(ry_test) * len(rz)
num_imgs_test = len(width) * len(d) * num_rots_test * iters_test

print("Generating %s images for training dataset, %s images for validation dataset, and %s images for testing dataset." % (num_imgs_train, num_imgs_valid, num_imgs_test))

Generating 16928 images for training dataset, 4232 images for validation dataset, and 4232 images for testing dataset.


Specifying the folder in which the images will be saved.

In [ ]:
folder_name = "Cubes Centred W %s-%s RX %s-%s RY %s-%s" % (int(width[0]), int(width[-1]), int(rx_train[0]), int(rx_train[-1]), int(ry_train[0]), int(ry_train[-1]))
dataset_path = os.path.join(cwd, folder_name)

print("Images will be saved to: %s." % folder_name)

Images will be saved to: Cubes Centred W 13-13 RX 0-45 RY 0-45.


The following function generates images for all combinations of widths, rotations and defects for a specified number of iterations. 

In [ ]:
def generate_imgs(shape_class, w, rx, ry, rz, a, iters, blur, save_path, file_type, aug):
  """
  Generates images of simulated nanoparticle with specified parameters and saves
  to a specified folder path.

  Inputs:
    shape_class: shape3D, lattice shape to generate
    w: Integer, width of lattice
    rx, ry, rz: List, list of Euler angles to rotate lattice
    a: Float, lattice spacing
    iters: Integer, number of iterations for each rotation
    blur: Integer, strength of Gaussian blur
    save_path: String, folder to save images
    file_type: String, file type to save images as
    aug: Boolean, whether or not to apply image augmentations (brightness & channel shift)
  """
  total_rotations = len(rx) * len(ry) * len(rz)
  total_images = len(w) * total_rotations * len(d) * iters
  counter = 1
  for width in w:
    for angle_z in rz:
      for angle_y in ry:
        for angle_x in rx:
          for i in range(iters):
            print("\rGenerating image %s of %s." % (counter, total_images), end='', flush=True)
            shape = shape_class(width, angle_x, angle_y, angle_z, a=a, defect=defect, pos_error=np.random.uniform(0.03, 0.045))
            shape.save_projection(save_path=save_path, file_type=file_type, blur=blur, iter=i, aug=aug)

              counter += 1

  print("\rGenerated %s images belonging to %s orientations (%s each)." % (total_images, total_rotations, int(total_images / total_rotations)), end='', flush=True)

  if aug:
    print(" Images were augmented so 3x the number of images were generated.")

Generating the training, validation, and testing images.

In [ ]:
begin_time = datetime.datetime.now()

# Generating the training images
save_path_train = os.path.join(dataset_path, "Train DR%s" % dr_train)

print("\nNow generating training dataset images...")
generate_imgs(
    shape_class=shape_class,
    w=width,
    rx=rx_train,
    ry=ry_train,
    rz=rz,
    a=a,
    d=d,
    iters=iters_train,
    blur=blur,
    save_path=save_path_train,
    file_type=file_type,
    aug=False
)

# Generating the validation images
save_path_valid = os.path.join(dataset_path, "Valid DR%s" % dr_train)

print("\nNow generating validation dataset images...")
generate_imgs(
  shape_class=shape_class,
  w=width,
  rx=rx_train,
  ry=ry_train,
  rz=rz,
  a=a,
  d=d,
  iters=iters_valid,
  blur=blur,
  save_path=save_path_valid,
  file_type=file_type,
  aug=False
)

# Generating the testing images
save_path_test = os.path.join(dataset_path, "Test DR%s" % dr_test)

print("\nNow generating testing dataset images...")
generate_imgs(
  shape_class=shape_class,
  w=width,
  rx=rx_test,
  ry=ry_test,
  rz=rz,
  a=a,
  d=d,
  iters=iters_test,
  blur=blur,
  save_path=save_path_test,
  file_type=file_type,
  aug=False
)

plt.close()

dt = datetime.datetime.now() - begin_time
dt_m = int(dt.total_seconds() // 60)
dt_s = int(dt.total_seconds() - (dt_m*60))
print("\n\nDone! The time it took is %sm %ss." % (dt_m, dt_s))


Now generating training dataset images...
Generated 16928 images belonging to 2116 orientations (8 each).
Now generating validation dataset images...
Generated 4232 images belonging to 2116 orientations (2 each).
Now generating testing dataset images...
Generated 4232 images belonging to 2116 orientations (2 each).

Done! The time it took is 38m 50s.


The specified parameters are saved into a text file.

In [ ]:
f = open(os.path.join(dataset_path, "Configuration.txt"), "w")
f.write("RX [%s, %s]\n" % (rx_train[0], rx_train[-1]))
f.write("RY [%s, %s]\n" % (ry_train[0], ry_train[-1]))
f.write("Width %s\n" % width)
f.write("a %s\n" % a)
f.write("d %s\n" % d)
f.write("blur %s\n" % blur)
f.write("iters %s" % iters)
f.close()